In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [3]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(901126, 2) Index(['split_str', 'org_idx'], dtype='object')
901126


In [4]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [5]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 901126/901126 [15:36:50<00:00, 16.03it/s]   


In [6]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['삼성', '카드', '특판', '적금', '서울', '행복', '신협', '구즉', '신협', '카카오']
['덮다', '몰다', '몰다', '감사하다', '나서다', '하다', '알다', '되다', '자다', '하다']
['소중하다', '적응하다', '귀찮다', '그렇다', '쏠쏠하다', '소소하다', '좋다', '있다', '좋아하다', '솔깃하다']
['서', '의', '의', '부터', '으로', '에', '로', '이야', '인가', '을']


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

151672


,명사,명사빈도
0,것,200062
1,수,123828
2,그,101065
3,이,96683
4,곳,67627
...,...,...
151667,독소제거,1
151668,광양제철선,1
151669,김호원,1
151670,복철,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1880


,동사,동사빈도
0,하다,855885
1,되다,177946
2,보다,104666
3,않다,81865
4,되어다,76175
...,...,...
1875,덩실대다,1
1876,젂다,1
1877,건덩거리다,1
1878,찌그러트리다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1858


,형용사,형용사빈도
0,있다,397840
1,없다,74254
2,좋다,67373
3,같다,63556
4,아니다,54286
...,...,...
1853,가당찮다,1
1854,반드레하다,1
1855,살살하다,1
1856,멀뚱멀뚱하다,1


In [10]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/7월/나무문화_7월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

396


,조사,조사빈도
0,을,690000
1,의,566133
2,이,523174
3,에,506282
4,를,336092
...,...,...
391,커녕은,1
392,보다만,1
393,에게다,1
394,에만은,1


In [11]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_7월_단어종합.txt", index=False, encoding='UTF-8')

In [12]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,0,"[25일, 급여, 삼성, 카드, 특판, 적금, 서울, 행복, 신협, 구즉, 신협, ...","[(25일, Number), (급여, Foreign), (삼성, Noun), (카드..."
1,근데 급 다음주부터 발령나서 다른 곳으로 출근한다,0,"[근데, 급, 다음주, 부터, 발령, 나서, 다른, 곳, 으로, 출근, 한다]","[(근데, Adverb), (급, Noun), (다음주, Noun), (부터, Jo..."
2,2 3일 전에 알려주고 갑자기 거기로 출근이야 좌천인가 걱정됨,0,"[2, 3일, 전, 에, 알려주고, 갑자기, 거기, 로, 출근, 이야, 좌천, 인가...","[(2, Number), (3일, Number), (전, Noun), (에, Jos..."
3,잘 적응하라고 뇌물을 준비해야 하나 ㅋㅋ 갑자기 문자메세지로 삼성카드 5월 링크 이...,0,"[잘, 적응하라고, 뇌물, 을, 준비, 해야, 하나, ㅋㅋ, 갑자기, 문자, 메세지...","[(자다, Verb), (적응하다, Adjective), (뇌물, Noun), (을..."
4,소소하게 가계부 적립할게 생겨서 기분 좋은 날 2 화요일 오래전 엄마 집 옆에 있는...,0,"[소소하게, 가계부, 적립, 할게, 생겨서, 기분, 좋은, 날, 2, 화요일, 오래...","[(소소하다, Adjective), (가계부, Noun), (적립, Noun), (..."
...,...,...,...,...
901121,현재 지하 1층에서 흙과몸 성긴 연결 촘촘한 관계 6 9 전시실 를 테마로 강요배 ...,18988,"[현재, 지하, 1, 층, 에서, 흙, 과, 몸, 성, 긴, 연결, 촘촘, 한, 관...","[(현재, Noun), (지하, Noun), (1, Number), (층, Noun..."
901122,그 외에 Big Man 다시 일어서는 몸 박치호 애도 상실의 끝에서 김수자 게르하르...,18988,"[그, 외, 에, Big, Man, 다시, 일어서는, 몸, 박치, 호, 애도, 상실...","[(그, Noun), (외, Noun), (에, Josa), (Big, Alpha)..."
901123,광양시 문화관광 061 797 3 광양시 관광안내소 광양 문화관광 1경 백운산 4대...,18988,"[광양시, 문화, 관광, 061, 797, 3, 광양시, 관광, 안내소, 광양, 문...","[(광양시, Noun), (문화, Noun), (관광, Noun), (061, Nu..."
901124,자막보기 공지사항 제11회 최수종과 함께하 동북아 LNG Hub 광양시립소년소녀합창...,18988,"[자막, 보기, 공지, 사항, 제, 11회, 최수종, 과, 함께하, 동북아, LNG...","[(자막, Noun), (보기, Noun), (공지, Noun), (사항, Noun..."


In [13]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_7월_형태소분석.txt", index=False, encoding='UTF-8')